<a href="https://colab.research.google.com/github/Crystal-Reshea/FinBert-Albert-nlp/blob/main/Sentiment_Analysis_with_Finbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis on Item 7

<b> Import Used </b>
* transformers
     * BertTokenizer
     * BertForSequenceClassification
     * pipeline
* pandas
* re
* nltk

In [ ]:
pip install transformers

     |████████████████████████████████| 3.3 MB 29.9 MB/s 
     |████████████████████████████████| 596 kB 44.0 MB/s 
     |████████████████████████████████| 3.3 MB 48.9 MB/s 
     |████████████████████████████████| 61 kB 423 kB/s 
     |████████████████████████████████| 895 kB 49.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import re
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3) # Bringing in FinBert model
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone') # Bringing in FinBert Tokenizer

nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer) # Creating Pipeline for finbert model


Downloading:   0%|          | 0.00/533 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/221k [00:00<?, ?B/s]

In [ ]:
def extract1(txt, start, end):
  assert start <= end, "Start should be less than or equal to end"
  lines = []
  i = 1
  flag = False
  with open(txt, 'r') as file: 
    for line in file:
        if end == i: 
          lines.append(line)
          break
        elif flag == True:
          lines.append(line)
        if start == i:
          flag = True
          lines.append(line)
        i+=1
  file.close()
  return ''.join(lines)

In [ ]:
# Setting up Test
file = '/content/bby-202110k.txt'
start=[1135,1161,1168,1172,1188,1235,1292,1297,1307,1311,1318,1322,1326,1340,1352,1365,1592,1610,1626,1646,1659,1673,1677,1682,1687,1692,1697,1719,1723,1737,1748,1753,1757,1761,1766,1770,1778,1783,1785,1788,1831,1837,1867,1877,1964,1971,1985,1991,1994,2002,2048,2099,2115]
end = [1158,1164,1168,1182,1229,1284,1293,1298,1307,1314,1318,1322,1336,1350,1359,1408,1604,1624,1644,1652,1670,1673,1678,1683,1688,1694,1712,1721,1735,1745,1749,1753,1757,1762,1766,1770,1780,1783,1786,1825,1835,1865,1871,1888,1967,1977,1988,1992,1995,2042,2093,2113,2142]

In [ ]:
# Collecting all sections of ITEM 7
paragraph = []
for i in range(len(start)): 
  paragraph.append(extract1(file, start[i],end[i]))

joined_paragraph = ''.join(paragraph)
print(joined_paragraph)

We are an omnichannel retailer that makes it easy for our customers to feel at home. We sell a wide assortment of merchandise in the Home, Baby, Beauty
& Wellness markets and operate under the names Bed Bath & Beyond ("BBB"), buybuy BABY ("BABY"), and Harmon, Harmon Face Values, or Face
Values (collectively, "Harmon"). We also operate Decorist, an online interior design platform that provides personalized home design services. In addition,
we are a partner in a joint venture, which operates retail stores in Mexico under the name Bed Bath & Beyond.
For fiscal 2020, 2019 and 2018, we accounted for our operations as two operating segments: North American Retail and Institutional Sales, the latter of
which did not meet the quantitative thresholds under GAAP and, therefore, was not a reportable segment. The Institutional Sales operating segment was
comprised of Linen Holdings, which was divested in October 2020. We will continue to account for our operations as one North American Retail rep

In [ ]:
# Cleaning ITEM 7 text to use for sentence tokenizing
joined_paragraph = joined_paragraph.replace('\n', ' ')
joined_paragraph_sentences = sent_tokenize(joined_paragraph)


In [ ]:
def remove_large_sent(): 
  index = 0
  to_remove = []
  for sentence in joined_paragraph_sentences: 
    if len(sentence) > 512: 
      to_remove.append(index)
    index+=1
  for i in range(len(to_remove)): 
    del joined_paragraph_sentences[(to_remove[i]-i)]
remove_large_sent()

In [ ]:
import pandas as pd
# Creating a dataframe of the collected sentiment scores for each sentence
def sentiment_score_df(arr): 
  sentence_col = []
  neutral_col = []
  positive_col = []
  negative_col = []
  for sentence in arr:
    sentiment_scores = nlp(sentence, return_all_scores=True)
    sentence_col.append(sentence)
    neutral_col.append(sentiment_scores[0][0]['score'])
    positive_col.append(sentiment_scores[0][1]['score'])
    negative_col.append(sentiment_scores[0][2]['score'])
  
  df = pd.DataFrame(list(zip(sentence_col, neutral_col, positive_col, negative_col)),
               columns =['sentence', 'neutral_score', 'positive_score', 'negative_score'])
  return df

In [ ]:
df = sentiment_score_df(joined_paragraph_sentences)

In [ ]:
df.head(10)

,sentence,neutral_score,positive_score,negative_score
0,We are an omnichannel retailer that makes it e...,4.513546e-03,9.954774e-01,9.087600e-06
1,We sell a wide assortment of merchandise in th...,9.999969e-01,9.625683e-07,2.197084e-06
2,"We also operate Decorist, an online interior d...",9.999437e-01,3.784461e-05,1.845245e-05
3,"In addition, we are a partner in a joint ventu...",9.999572e-01,9.241641e-06,3.350702e-05
4,"For fiscal 2020, 2019 and 2018, we accounted f...",9.999901e-01,2.239554e-06,7.680431e-06
5,The Institutional Sales operating segment was ...,9.999588e-01,6.057570e-07,4.059985e-05
6,We will continue to account for our operations...,9.999963e-01,2.530398e-07,3.426756e-06
7,We have undertaken significant changes over th...,9.978219e-01,1.868782e-03,3.093244e-04
8,"During the past year, as the world responded t...",6.427469e-05,9.993356e-01,6.000464e-04
9,"Similar to many other businesses, the COVID-19...",9.211252e-09,1.000000e+00,5.561910e-09


In [ ]:
print(df['neutral_score'].mean())
print(df['positive_score'].mean())
print(df['negative_score'].mean())

0.7996008021295594
0.08542136562516359
0.11497783637192334


# Question Answering using BERT - w/o fine-tuning

<b>Imports used</b>
* torch
* transformers
     * BertForQuestionAnsering
     * BertTokenizer


In [ ]:
import torch 
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
 
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
sample_txt = joined_paragraph[:2473]
question="What changes were made due to covid-19?"

In [ ]:
joined_paragraph[:2551]

'We are an omnichannel retailer that makes it easy for our customers to feel at home. We sell a wide assortment of merchandise in the Home, Baby, Beauty & Wellness markets and operate under the names Bed Bath & Beyond ("BBB"), buybuy BABY ("BABY"), and Harmon, Harmon Face Values, or Face Values (collectively, "Harmon"). We also operate Decorist, an online interior design platform that provides personalized home design services. In addition, we are a partner in a joint venture, which operates retail stores in Mexico under the name Bed Bath & Beyond. For fiscal 2020, 2019 and 2018, we accounted for our operations as two operating segments: North American Retail and Institutional Sales, the latter of which did not meet the quantitative thresholds under GAAP and, therefore, was not a reportable segment. The Institutional Sales operating segment was comprised of Linen Holdings, which was divested in October 2020. We will continue to account for our operations as one North American Retail re

In [ ]:
def answerGenerator(question, text): 
  input_ids = tokenizer.encode(question, sample_txt)
  tokens = tokenizer.convert_ids_to_tokens(input_ids)

  sep_index = input_ids.index(tokenizer.sep_token_id)
  num_seg_a = sep_index + 1
  num_seb_b = len(input_ids) - num_seg_a
  segment_ids = [0]*num_seg_a + [1]*num_seb_b
  assert len(segment_ids) == len(input_ids)

  start_scores, end_scores = model(torch.tensor([input_ids]),
                                 token_type_ids=torch.tensor([segment_ids]), return_dict=False)
  
  answer_start = torch.argmax(start_scores)
  answer_end = torch.argmax(end_scores)
  tokens = tokenizer.convert_ids_to_tokens(input_ids)
  answer = tokens[answer_start]

  for i in range(answer_start + 1, answer_end + 1): 
    if tokens[i][0:2] =='##': 
      answer==tokens[i][2:]
    else: 
      answer += ' ' + tokens[i]

  return('Answer: "' + answer + '"')


In [ ]:
question="What changes were made due to covid-19?"
answerGenerator(question, sample_txt)

'Answer: "extensive changes to executive leadership , as well as development of essential strategies and plans to focus on positioning our business for long - term success"'